# Importing libraries

In [246]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import statistics as stats
import matplotlib.pyplot as plt
import tensorflow as tf

# Importing dataset

In [247]:
# train_dataset = pd.read_csv('../input/titanic/train.csv')
train_dataset = pd.read_csv('train.csv')
# print('for TRAINING SET : \n',train_dataset)
train_dataset.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [248]:
# test_dataset = pd.read_csv('test.csv')
# test_dataset = pd.read_csv('../input/titanic/test.csv')
# print('for TESTING SET : \n',test_dataset)

# test_dataset = pd.concat(map(pd.read_csv, ['gender_submission.csv', 'test.csv']), ignore_index=True)

test_dataset_X = pd.read_csv('test.csv')
test_dataset_y = pd.read_csv('gender_submission.csv')
test_dataset = pd.merge(test_dataset_y,test_dataset_X, on='PassengerId', how='inner')
test_dataset.head(10)
# print(len(test_dataset.values))

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
5,897,0,3,"Svensson, Mr. Johan Cervin",male,14.0,0,0,7538,9.2250,NaN,S
6,898,1,3,"Connolly, Miss. Kate",female,30.0,0,0,330972,7.6292,NaN,Q
7,899,0,2,"Caldwell, Mr. Albert Francis",male,26.0,1,1,248738,29.0000,NaN,S
8,900,1,3,"Abrahim, Mrs. Joseph (Sophie Halaut Easu)",female,18.0,0,0,2657,7.2292,NaN,C
9,901,0,3,"Davies, Mr. John Samuel",male,21.0,2,0,A/4 48871,24.1500,NaN,S


# Preprocessing

In [249]:
# # Checking `null` values in dataset ; finding null values in columns
print('for TRAINING SET : \n',train_dataset.isnull().sum())
print('\nfor TESTING SET : \n',test_dataset.isnull().sum())

for TRAINING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

for TESTING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


> Missing values found : 
- training set : age, embarked
- testing set : age, cabin

## Filling missing values

- `Age` : 

In [250]:
train_dataset['Age'].fillna(train_data['Age'].mean(), inplace=True)
print('for TRAINING SET : \n',train_dataset.isnull().sum())
test_dataset['Age'].fillna(train_data['Age'].mean(), inplace=True)
print('for TRAINING SET : \n',test_dataset.isnull().sum())

for TRAINING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64
for TRAINING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64


- `Embarked` : 

In [251]:
train_dataset['Embarked'].fillna(train_data['Embarked'].mode()[0], inplace=True)
print('for TRAINING SET : \n',train_dataset.isnull().sum())

for TRAINING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         0
dtype: int64


- `Fare` : 

In [252]:
test_dataset['Fare'].fillna(train_data['Fare'].mean(), inplace=True)
print('for TRAINING SET : \n',test_dataset.isnull().sum())

for TRAINING SET : 
 PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          327
Embarked         0
dtype: int64


> considering only : `PClass`, `Sex`, `Age`, `SibSp`, `Parch`, `Fare`, `Embarked`

In [253]:
X_train = train_dataset.iloc[:,[2,4,5,6,7,9,11]].values
# X_train = train_dataset.iloc[:,[2,4,5,6,7,9,11]]
X_test = test_dataset.iloc[:,[2,4,5,6,7,9,11]].values
# X_test = test_dataset.iloc[:,[1,3,4,5,6,8,10]]
y_train = train_dataset.iloc[:,1].values
# y = train_dataset.iloc[:,1]
y_test = test_dataset.iloc[:,1].values

In [254]:
X_train

array([[3, 'male', 22.0, ..., 0, 7.25, 'S'],
       [1, 'female', 38.0, ..., 0, 71.2833, 'C'],
       [3, 'female', 26.0, ..., 0, 7.925, 'S'],
       ...,
       [3, 'female', 29.69911764705882, ..., 2, 23.45, 'S'],
       [1, 'male', 26.0, ..., 0, 30.0, 'C'],
       [3, 'male', 32.0, ..., 0, 7.75, 'Q']], dtype=object)

In [255]:
X_test

array([[3, 'male', 34.5, ..., 0, 7.8292, 'Q'],
       [3, 'female', 47.0, ..., 0, 7.0, 'S'],
       [2, 'male', 62.0, ..., 0, 9.6875, 'Q'],
       ...,
       [3, 'male', 38.5, ..., 0, 7.25, 'S'],
       [3, 'male', 29.69911764705882, ..., 0, 8.05, 'S'],
       [3, 'male', 29.69911764705882, ..., 1, 22.3583, 'C']], dtype=object)

In [256]:
y_train

array([0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1,

In [257]:
y_test

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [258]:
# before encoding : 
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

[3 'male' 22.0 1 0 7.25 'S']
0
[3 'male' 34.5 0 0 7.8292 'Q']
0


## Encoding categorical data

- encoding `gender` : 

In [259]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X_train[:, 1] = le.fit_transform(X_train[:, 1])
X_test[:, 1] = le.fit_transform(X_test[:, 1])

In [260]:
# after encoding gender : 
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

[3 1 22.0 1 0 7.25 'S']
0
[3 1 34.5 0 0 7.8292 'Q']
0


- encoding `Embarked`

In [261]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [6])], remainder='passthrough')
X_train = np.array(ct.fit_transform(X_train))
X_test = np.array(ct.fit_transform(X_test))

In [262]:
# after encoding embarked : 
print(X_train[0])
print(y_train[0])
print(X_test[0])
print(y_test[0])

[0.0 0.0 1.0 3 1 22.0 1 0 7.25]
0
[0.0 1.0 0.0 3 1 34.5 0 0 7.8292]
0


# Feature Scaling

In [263]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)   # only transforming to avoid Information Leakage

# Building the ANN

## 1. Initializing ANN

In [264]:
ann = tf.keras.models.Sequential()

## 2. Adding theinputlayer and the first hidden layer
- `units` : number of neurons in the layer
- `relu` : rectifier linear unit

In [265]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## 3. Adding the second hidden layer

In [266]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

## 4. Adding the output layer
- as dimension of the output is 1 *(whether person is survived or not)*
- choosing sigmoid activation as we have binary classification

In [267]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Training the ANN

## 1. compiling the ANN
- here we are Updating weights and reducing cost.
- `optimizer` : gradient descent method. Here we are choosing adam as one of the method.
- `loss` : as we are doing binary classification, we'll use binary_crossentropy which will apply CROSS ENTROPY eq.
For non-binary classification use categorical_crossentropy as loss function and softmax as activation function above.
- `metrics` : choosing accuracy. although we can also choose multiple metrices.

In [268]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## 2. Training the ANN on the training set

In [269]:
ann.fit(X_train, y_train, batch_size=32, epochs=100)

Epoch 1/100
28/28 [==============================] - 1s 2ms/step - loss: 0.6759 - accuracy: 0.6296
Epoch 2/100
28/28 [==============================] - 0s 1ms/step - loss: 0.6377 - accuracy: 0.6779
Epoch 3/100
28/28 [==============================] - 0s 3ms/step - loss: 0.6067 - accuracy: 0.6947
Epoch 4/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5792 - accuracy: 0.7172
Epoch 5/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5531 - accuracy: 0.7284
Epoch 6/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5278 - accuracy: 0.7576
Epoch 7/100
28/28 [==============================] - 0s 2ms/step - loss: 0.5064 - accuracy: 0.7778
Epoch 8/100
28/28 [==============================] - 0s 3ms/step - loss: 0.4895 - accuracy: 0.7890
Epoch 9/100
28/28 [==============================] - 0s 2ms/step - loss: 0.4778 - accuracy: 0.7890
Epoch 10/100
28/28 [==============================] - 0s 2ms/step - loss: 0.4693 - accuracy: 0.7969
Epoch 11/

# Making prediction and Model evaluation
- Prediction

In [283]:
y_pred = (ann.predict(X_test)>0.5)

- Evaluation through metrices

In [278]:
from sklearn.metrics import confusion_matrix, accuracy_score
print('Confusion Matrix :\n', confusion_matrix(y_test, y_pred))
print('Accuracy Score :', accuracy_score(y_test, y_pred))

Confusion Matrix :
 [[247  19]
 [ 34 118]]
Accuracy Score : 0.8732057416267942


- Making csv file for submission

In [284]:
test_data.PassengerId

0       892
1       893
2       894
3       895
4       896
       ... 
413    1305
414    1306
415    1307
416    1308
417    1309
Name: PassengerId, Length: 418, dtype: int64

In [288]:
predictions = []
for i in y_pred:
  if i[0]:
    predictions.append(1)
  else:
    predictions.append(0)
predictions

[0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,


In [289]:
output = pd.DataFrame({"PassengerId": test_data.PassengerId, "Survived": predictions})
print(output)
output.to_csv('19BCE245_DL_Prac1.csv', index=False)

     PassengerId  Survived
0            892         0
1            893         0
2            894         0
3            895         0
4            896         0
..           ...       ...
413         1305         0
414         1306         1
415         1307         0
416         1308         0
417         1309         0

[418 rows x 2 columns]
